In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
%matplotlib inline
images=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images")
a=os.listdir(images)
a.sort()
test_images=a[:1698]
train_images=a[1698:]

In [ ]:
train_csv=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/train.csv"))
train_csv=train_csv.sort_values(by=['name'])
submission=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/submission.csv"))
# codes={}
# train_csv["classname"].unique()
# for i in range(len(df["class"].unique())):
#     codes[df["class"].unique()[i]]=train_csv["classname"].unique()[i]
# print(codes)


In [ ]:
df=train_csv
bbox=[]
x=[]
y=[]
w=[]
h=[]
for i in range(len(train_csv)):
    arr=[]
    for j in df.iloc[i][["x1",'x2','y1','y2']]:
        arr.append(j)
    arr[2]=arr[2]-arr[0]
    arr[3]=arr[3]-arr[1]
    bbox.append(arr)
    x.append(arr[0])
    y.append(arr[1])
    w.append(arr[2])
    h.append(arr[3])
df["bbox"]=bbox
df["x"]=x
df["y"]=y
df["w"]=w
df["h"]=h
df.classname = pd.Categorical(df.classname)
df["class"] = df.classname.cat.codes
df.drop(columns=["x1","x2",'y1','y2'],inplace=True)
df.columns=["image_id","classname","bbox","x","y","w","h","class"]
submission["image_id"]=submission["name"]
df.drop(columns="classname",inplace=True)
df[["x","y","w","h"]]=df[["x","y","w","h"]].astype(float)
df.bbox=df.bbox.astype(str)
df.head()

In [ ]:
# codes={}
# train_csv["classname"].unique()
# for i in range(len(df["class"].unique())):
#     codes[df["class"].unique()[i]]=train_csv["classname"].unique()[i]
# print(codes)

In [ ]:
def get_boxes(id):
    boxes=[]
    for i in df[df["image_id"]==str(id)]["bbox"]:
        boxes.append(list(map(float,i[1:-1].split(", "))))
    return boxes
print(get_boxes('1806.jpg'))

In [ ]:
import matplotlib.pyplot as plt
import cv2
import matplotlib.patches as patches
image="1799.jpg"

img=plt.imread(os.path.join(images,image))

fig,ax = plt.subplots(1)
ax.imshow(img)
boxes=get_boxes(image)
for box in boxes:
    rect = patches.Rectangle((box[0],box[1]),box[2],box[3],linewidth=2,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
plt.show()

In [ ]:
boxes

In [ ]:
import cv2
import os
import re
import torch
import torchvision
from torchvision import transforms 
import numpy as np
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from matplotlib import pyplot as plt

In [ ]:
trans = transforms.Compose([transforms.ToTensor()])

In [ ]:
class WheatDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None,train=True):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        self.train=train

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms is not None:  #Apply transformation
            image = self.transforms(image)
        if(self.train==False):  # For test data
            return image, image_id
        #Else for train and validation data
        records = self.df[self.df['image_id'] == image_id]   
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.as_tensor(list(records["class"]), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        return image, target,image_id  

In [ ]:
class Averager:      ##Return the average loss 
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
        
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = WheatDataset(df, images, trans,True)
test_dataset = WheatDataset(submission, images, trans,False)



# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)
test_data_loader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# img,image_ids= next(iter(test_data_loader))
# print(img[1].shape)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# images, targets, image_ids = next(iter(train_data_loader))
# images = list(image.to(device) for image in images)
# targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

# boxes = targets[3]['boxes'].cpu().numpy().astype(np.int32)
# sample = images[3].permute(1,2,0).cpu().numpy()

# fig, ax = plt.subplots(1, 1, figsize=(16, 8))

# for box in boxes:
#     cv2.rectangle(sample,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (220, 0, 0), 3)
    
# ax.set_axis_off()
# ax.imshow(sample)

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, pretrained_backbone=False)
num_classes = 21  # 1 class (wheat) + background

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [ ]:
pass_this=('1861.jpg', '1862.jpg', '1864.jpg', '1865.jpg')

In [ ]:
import math
model.train()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
#lr_scheduler = None

num_epochs = 5

loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader:
        if image_ids==pass_this:
            continue
        images = list(image.to(device) for image in images)
#         if itr==1:    
#             print("images")
#             print(images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#         if itr==1:
#             print("targets")
#             print(targets)
        loss_dict = model(images, targets)   ##Return the loss
#         print("loss dict")
#         print(loss_dict)
        losses = sum(loss for loss in loss_dict.values())
        x=float(losses)
#         print(f"losses {losses.item()} and {loss_hist.value}")
        if math.isnan(x):
            print(images,targets)
            break
        loss_value = losses.item()
#         print("loss_value ",loss_value)
        loss_hist.send(loss_value)  #Average out the loss

    
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 100 == 0:
            print(f"Iteration #{itr} loss: {loss_value} losss: {losses}")

        itr += 1
#         if itr==2:
#             break
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value} losss:{loss_value}")
#     break

In [ ]:
def format_prediction_string(boxes, scores,labels): ## Define the formate for storing prediction results
#     pred_strings = []
#     for j in zip(scores, boxes,labels):
# #         print(j)
#         pred_strings.append("{0} {1} {2} {3} {4}".format(j[1][0], j[1][1], j[1][2], j[1][3],j[2]))

#     return " ".join(pred_strings)
    box=[]
    lab=[]
    for j in zip(scores,boxes,labels):
        box.append([j[1][0], j[1][1], j[1][2], j[1][3]])
        lab.append(j[2])
    return box,lab

In [ ]:
## Lets make the prediction
results=[]
model.eval()
detection_threshold = 0.40

for images, image_ids in test_data_loader:    

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()    ##Formate of the output's box is [Xmin,Ymin,Xmax,Ymax]
        scores = outputs[i]['scores'].data.cpu().numpy()
        labels = outputs[i]['labels'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32) #Compare the score of output with the threshold and
        labels = labels[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]                 

        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]         
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]         #Convert the box formate to [Xmin,Ymin,W,H]
        
        
        b,l=format_prediction_string(boxes, scores,labels)    
        result = {                                     #Store the image id and boxes and scores in result dict.
            'image_id': image_id,
#             'PredictionString': format_prediction_string(boxes, scores,labels)
            'boxes':b,
            'labels':l
        }

        
        results.append(result)              #Append the result dict to Results list

In [ ]:
train_csv=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/train.csv"))
train_csv=train_csv.sort_values(by=['name'])
submission=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/submission.csv"))
codes={}
train_csv["classname"].unique()
for i in range(len(df["class"].unique())):
    codes[df["class"].unique()[i]]=train_csv["classname"].unique()[i]
print(codes)


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
%matplotlib inline
images=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images")
a=os.listdir(images)
a.sort()
test_images=a[:1698]
train_images=a[1698:]
import matplotlib.pyplot as plt
import cv2
import matplotlib.patches as patches
image="1800.jpg"

img=plt.imread(os.path.join(images,image))

fig,ax = plt.subplots(1)
ax.imshow(img)
boxes=get_boxes(image)
# for i in results[0]["boxes"]:
#     rect = patches.Rectangle((i[0],i[1]),i[2],i[3],linewidth=2,edgecolor='r',facecolor='none')
#     ax.add_patch(rect)
rect = patches.Rectangle((913,252),575,631,linewidth=2,edgecolor='r',facecolor='none')
ax.add_patch(rect)
# rect = patches.Rectangle((328,391),156,140,linewidth=2,edgecolor='r',facecolor='none')
# ax.add_patch(rect)
# rect = patches.Rectangle((906,185),107,60,linewidth=2,edgecolor='r',facecolor='none')
# ax.add_patch(rect)
# rect = patches.Rectangle((675,203),175,225,linewidth=2,edgecolor='r',facecolor='none')
# ax.add_patch(rect)
plt.show()

In [ ]:
print(i)

In [ ]:
results[2]["image_id"]

In [ ]:
test_df = pd.DataFrame(columns=['name', "x1","x2","y1","y2","classname"])
for i in range(len(results)):
    for j in range(len(results[i]["boxes"])):
        test_df=test_df.append({'name':results[i]["image_id"], "x1":results[i]["boxes"][j][0],"x2":results[i]["boxes"][j][1],"y1":results[i]["boxes"][j][2]+results[i]["boxes"][j][0],"y2":results[i]["boxes"][j][3]+results[i]["boxes"][j][1],"classname":codes[results[i]["labels"][j]]},ignore_index=True)
test_df.head()

In [ ]:
test_df.head(25)

In [ ]:
test_df.to_csv('submission.csv', index=False)
